In [16]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.metrics import classification_report_imbalanced



---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [17]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv(
    Path('./Resources/lending_data.csv')
)
lending_data_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [18]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = lending_data_df['loan_status'] 
# Separate the X variable, the features
X = lending_data_df.drop('loan_status', axis=1) 

In [19]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [20]:
# Review the X variable DataFrame
X[:5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [21]:
# Check the balance of our target values
label_counts = y.value_counts()
print(label_counts)


0    75036
1     2500
Name: loan_status, dtype: int64


### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [45]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)



---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [23]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs',
                                 max_iter=200,
                                 random_state=1)
classifier

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [35]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Predictions": predictions, "Actual": y_test}).reset_index(drop=True)

results.head(100)

,Predictions,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
95,0,0
96,0,0
97,0,0
98,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [25]:
# Print the balanced_accuracy score of the model
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, predictions)

cm_oversampled = confusion_matrix(y_test, predictions)
cm_oversampled_df = pd.DataFrame(cm_oversampled, 
                                index = ['Actual Healthy Loans (low-risk)', 
                                'Actual Non-Healthy Loans (high-risk)'], 
                                columns = ['Predicted Healthy Loans (low-risk)', 'Predicted Non-Healthy Loans (high-risk)']
                              )
cm_oversampled_df


,Predicted Healthy Loans (low-risk),Predicted Non-Healthy Loans (high-risk)
Actual Healthy Loans (low-risk),18663,102
Actual Non-Healthy Loans (high-risk),56,563


In [26]:
# Print the classification report for the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**  The cm_oversampled_df report has a 99% accuracy when identifying a low risk loan and 100% accuracy in not identifying a high risk loan as a low risk loan.
    Identifying the High risk loans has a lower accuracy of 91% with an 85% precision score.
    This will easily identify low risk loans but the high risk loans seem to be a more difficult to identify and may take more investigation to accurately identify.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [27]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification
from collections import Counter

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
X, y = make_classification(n_samples=100, n_features=10, n_informative=2, n_classes=2, random_state=1)
print("Original label distribution:", Counter(y))

# Fit the original training data to the random_oversampler model
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)
print("Resampled label distribution:", Counter(y_resampled))


Original label distribution: Counter({0: 51, 1: 49})
Resampled label distribution: Counter({1: 51, 0: 51})


In [29]:
# Count the distinct values of the resampled labels data
distinct_values = np.unique(X_resampled)
count = len(distinct_values)

print("Distinct values:", distinct_values)
print("Count:", count)

Distinct values: [-2.79308500e+00 -2.79099641e+00 -2.78253447e+00 -2.69836174e+00
 -2.51630386e+00 -2.50644065e+00 -2.41350300e+00 -2.30490794e+00
 -2.23708651e+00 -2.22711263e+00 -2.14444405e+00 -2.14246673e+00
 -2.11416392e+00 -2.09424782e+00 -2.08639057e+00 -2.07680202e+00
 -2.03720123e+00 -1.95816909e+00 -1.95210529e+00 -1.94504696e+00
 -1.94332341e+00 -1.94258918e+00 -1.94068096e+00 -1.89881206e+00
 -1.89526695e+00 -1.88141087e+00 -1.87316098e+00 -1.82391985e+00
 -1.78962393e+00 -1.78791289e+00 -1.77756638e+00 -1.75592564e+00
 -1.75151751e+00 -1.73440213e+00 -1.71939447e+00 -1.69933360e+00
 -1.68405999e+00 -1.67419581e+00 -1.66153635e+00 -1.63971434e+00
 -1.63744959e+00 -1.57921328e+00 -1.57915629e+00 -1.56917330e+00
 -1.56629795e+00 -1.56573815e+00 -1.56132059e+00 -1.55694156e+00
 -1.54398637e+00 -1.53879325e+00 -1.52568032e+00 -1.52212235e+00
 -1.51649333e+00 -1.51594823e+00 -1.51555340e+00 -1.51045638e+00
 -1.50957268e+00 -1.49363676e+00 -1.48515645e+00 -1.47955308e+00
 -1.4791

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [37]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier_resampled = LogisticRegression(solver='lbfgs',
                                 max_iter=200,
                                 random_state=1)
classifier_resampled

# Fit the model using the resampled training data
classifier_resampled.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
predictions = classifier_resampled.predict(X_resampled)
results_resampled = pd.DataFrame({"Predictions": predictions, "Actual": y_resampled}).reset_index(drop=True)

results_resampled.head()

,Predictions,Actual
0,1,1
1,0,0
2,1,1
3,0,0
4,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [38]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_resampled, predictions)

0.9705882352941176

In [40]:
# Generate a confusion matrix for the model
cm_Lr_oversampled = confusion_matrix(y_resampled, predictions)
cm_Lr_oversampled_df = pd.DataFrame(cm_Lr_oversampled, 
                                index = ['Actual Healthy Loans (low-risk)', 
                                'Actual Non-Healthy Loans (high-risk)'], 
                                columns = ['Predicted Healthy Loans (low-risk)', 'Predicted Non-Healthy Loans (high-risk)']
                              )
cm_Lr_oversampled_df


,Predicted Healthy Loans (low-risk),Predicted Non-Healthy Loans (high-risk)
Actual Healthy Loans (low-risk),50,1
Actual Non-Healthy Loans (high-risk),2,49


In [41]:
# Print the classification report for the model
print(classification_report(y_resampled, predictions))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97        51
           1       0.98      0.96      0.97        51

    accuracy                           0.97       102
   macro avg       0.97      0.97      0.97       102
weighted avg       0.97      0.97      0.97       102



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** There is a much better balance of the data now.  Both high and low risk loans are identified with 98-96% accuracy and the precision of the data is 96-98%.  
    This appears to be a very good evaluation of high and low risk loans.